In [1]:
p_config = { # Trigger는 다시 당겨지고..
    "asic_mode":                      {"trigger": 1,  "value": 0},   # 0 training_only, 1 train_inf_sweep, 2 inference_only
    "training_epochs":                {"trigger": 2,  "value": 200},
    "inference_epochs":               {"trigger": 3,  "value": 500},
    "dataset":                        {"trigger": 4,  "value": 1},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
    "timesteps":                      {"trigger": 5,  "value": 5},
    "input_size_layer1":              {"trigger": 6,  "value": 578},
    "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
    "binary_classifier_mode":         {"trigger": 8,  "value": 0},
    "loser_encourage_mode":           {"trigger": 9,  "value": 0},
    "layer1_cut_list":                {"trigger": 10, "value": [256,17,171,342,342,342,342,342,342,342,342,342,342,342,496]},
    "layer2_cut_list":                {"trigger": 11, "value": [256,17,171,342,342,342,342,342,342,342,342,342,342,342,496]},
}

import time
import numpy as np
from mms_ok import XEM7310
from secrets import token_hex

def signed_encoding_32bit(value):
    return value & 0xFFFFFFFF
def signed_decoding_32bit(value):
    return value - 0x100000000 if value & 0x80000000 else value

bitstream_path = r"bitstream/top_bh_fpga.bit"
fpga = XEM7310(bitstream_path=bitstream_path)


2026-01-06 20:27:49.500 | INFO     | mms_ok.ok_setup:copy_frontpanel_files:28 - FrontPanel SDK Version: 5.3.6
2026-01-06 20:27:49.505 | INFO     | mms_ok.ok_setup:copy_frontpanel_files:39 - FrontPanel API ready
2026-01-06 20:27:49.665 | INFO     | mms_ok.fpga:_validate_bitstream_path:117 - Bitstream file: c:\Users\User\measurement_setting\bitstream\top_bh_fpga.bit
2026-01-06 20:27:49.665 | INFO     | mms_ok.fpga:_validate_bitstream_path:120 - Bitstream date: 2026-01-06 20:25:09
2026-01-06 20:27:49.671 | INFO     | mms_ok.fpga:_connect:147 - Model        : XEM7310-A75
2026-01-06 20:27:49.672 | INFO     | mms_ok.fpga:_connect:148 - Serial Number: 1917000Q6P
2026-01-06 20:27:49.672 | INFO     | mms_ok.fpga:_connect:149 - Interface    : USB 3
2026-01-06 20:27:49.672 | INFO     | mms_ok.fpga:_connect:150 - USB Speed    : SUPER
2026-01-06 20:27:49.673 | INFO     | mms_ok.fpga:_connect:151 - Max Blocksize: 16384
2026-01-06 20:27:49.673 | INFO     | mms_ok.fpga:_connect:152 - Wire Width   : 32

In [2]:
# ######### Reset ######################################################################
reset_time = 1.0  # seconds
print(f'Resetting FPGA for {reset_time} seconds...')
fpga.reset(reset_address = 0x00, reset_time = reset_time, active_low = True)
print('System reset complete.')
time.sleep(1.0) # 이거 꼭 해라
fpga.CheckTriggered(0x60, 1, timeout=10.0) # config 다됐냐
# ######### Reset ######################################################################

print('')
print('')
print('')
print('')

# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'WireOut 0x20 value: {ep20wireout}')
# ep21wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x21))
# print(f'WireOut 0x21 value: {ep21wireout}')

# ######### p_config mode ######################################################################
p_config_ongoing_sleep_time = 0.01  # seconds
p_config_verbose = False
# config mode on
print(f'######### p_config mode ######################################################################')
fpga.SetWireInValue(0x01, signed_encoding_32bit(1))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# asic_mode: 0 training_only, 1 train_inf_sweep, 2 inference_only 
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['asic_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['asic_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['asic_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- asic_mode')
time.sleep(p_config_ongoing_sleep_time)
# training_epochs
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['training_epochs']['value']))
fpga.ActivateTriggerIn(0x40, p_config['training_epochs']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['training_epochs']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- training_epochs')
time.sleep(p_config_ongoing_sleep_time)
# inference_epochs
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['inference_epochs']['value']))
fpga.ActivateTriggerIn(0x40, p_config['inference_epochs']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['inference_epochs']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- inference_epochs')
time.sleep(p_config_ongoing_sleep_time)
# dataset: 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['dataset']['value']))
fpga.ActivateTriggerIn(0x40, p_config['dataset']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['dataset']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- dataset')
time.sleep(p_config_ongoing_sleep_time)
# timesteps
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['timesteps']['value']))
fpga.ActivateTriggerIn(0x40, p_config['timesteps']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['timesteps']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- timesteps')
time.sleep(p_config_ongoing_sleep_time)
# input_size_layer1_define
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['input_size_layer1']['value']))
fpga.ActivateTriggerIn(0x40, p_config['input_size_layer1']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['input_size_layer1']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- input_size_layer1')
time.sleep(p_config_ongoing_sleep_time)
# long_time_input_streaming_mode
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['long_time_input_streaming_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['long_time_input_streaming_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['long_time_input_streaming_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- long_time_input_streaming_mode')
time.sleep(p_config_ongoing_sleep_time)
# binary_classifier_mode
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['binary_classifier_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['binary_classifier_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['binary_classifier_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- binary_classifier_mode')
time.sleep(p_config_ongoing_sleep_time)
# loser_encourage_mode
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['loser_encourage_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['loser_encourage_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['loser_encourage_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- loser_encourage_mode')
time.sleep(p_config_ongoing_sleep_time)
# layer1_cut_list
for i in range(15):
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer1_cut_list']['value'][i]))
    fpga.ActivateTriggerIn(0x40, p_config['layer1_cut_list']['trigger'])
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer1_cut_list']['trigger']))
    time.sleep(p_config_ongoing_sleep_time)
    ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
    if p_config_verbose:
        print(f'WireOut 0x20 value: {ep20wireout} <-- layer1_cut_list[{i}]')
# layer2_cut_list
for i in range(15):
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer2_cut_list']['value'][i]))
    fpga.ActivateTriggerIn(0x40, p_config['layer2_cut_list']['trigger'])
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer2_cut_list']['trigger']))
    time.sleep(p_config_ongoing_sleep_time)
    ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
    if p_config_verbose:
        print(f'WireOut 0x20 value: {ep20wireout} <-- layer2_cut_list[{i}]')
# ######### p_config mode######################################################################
        
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))

fpga.ActivateTriggerIn(0x40, 31)

fpga.CheckTriggered(0x60, 1, timeout=10.0) # config 다됐냐




    
print('')
print('')
print('')
print('')

# ######### p_config done mode ######################################################################
print(f'######### p_config done mode ######################################################################')
fpga.SetWireInValue(0x01, signed_encoding_32bit(2))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# ######### p_config done mode ######################################################################


    
print('')
print('')
print('')
print('')



# ######### P_STATE_03_DRAMFILL_WEIGHT_DATA mode ######################################################################
print(f'######### P_STATE_03_DRAMFILL_WEIGHT_DATA mode ######################################################################')
fpga.SetWireInValue(0x01, signed_encoding_32bit(3))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))


fpga.SetWireInValue(0x01, signed_encoding_32bit(8)) # dram write address
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(16)) # dram write address last
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
fpga.CheckTriggered(0x60, 1, timeout=10.0) # config 다됐냐


data = token_hex(64) # byte
print(f"Data: {data} [{type(data)}]")

transfer_byte = fpga.WriteToBlockPipeIn(0x80, data, reorder_str=True)
print(f"Transfer byte: {transfer_byte} bytes")

fpga.CheckTriggered(0x60, 1, timeout=10.0) # config 다됐냐
# ######### P_STATE_03_DRAMFILL_WEIGHT_DATA mode ######################################################################



# ######### P_STATE_04_DRAMFILL_WEIGHT_DATA_DONE mode ######################################################################
# fpga.SetWireInValue(0x01, signed_encoding_32bit(1))
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# ######### P_STATE_04_DRAMFILL_WEIGHT_DATA_DONE mode ######################################################################






# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'WireOut 0x20 value: {ep20wireout}')
# ep21wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x21))
# print(f'WireOut 0x21 value: {ep21wireout}')


2026-01-06 20:27:49.873 | INFO     | mms_ok.fpga:reset:264 - Reset Start (1.0s)


Resetting FPGA for 1.0 seconds...


2026-01-06 20:27:50.885 | INFO     | mms_ok.fpga:reset:272 - Reset End (1.0s)



System reset complete.




######### p_config mode ######################################################################


2026-01-06 20:28:02.487 | ERROR    | mms_ok.fpga:CheckTriggered:552 - Trigger (0x60) condition not met within 10.0s


TimeoutError: Trigger (0x60) condition not met within timeout

In [6]:
# fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer2_cut_list']['trigger']))
fpga.SetWireInValue(0x01, 11)

ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout}')
ep21wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x21))
print(f'WireOut 0x21 value: {ep21wireout}')

WireOut 0x20 value: 1
WireOut 0x21 value: 1


In [ ]:
# # fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer2_cut_list']['trigger']))
# fpga.SetWireInValue(0x01, 0)

# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'WireOut 0x20 value: {ep20wireout}')
# ep21wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x21))
# print(f'WireOut 0x21 value: {ep21wireout}')

WireOut 0x20 value: 0
WireOut 0x21 value: 1


In [ ]:
# fpga.close() 